In [1]:
import torch
import torch.nn as nn

In [ ]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMCell, self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size

        #Input Gate components
        self.W_ii=nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.W_hi=nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_i=nn.Parameter(torch.Tensor(hidden_size))

        #Forget Gate components
        self.W_if=nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.W_hf=nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_f=nn.Parameter(torch.Tensor(hidden_size))

        #Cell Gate components
        self.W_ig=nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.W_hg=nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_g=nn.Parameter(torch.Tensor(hidden_size))

        #Output Gate components
        self.W_io=nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.W_ho=nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_o=nn.Parameter(torch.Tensor(hidden_size))

        self.init_weights()

    def init_weights(self):
        for param in self.parameters():
            #가중치를 -0.1 ~ 0.1 사이의 값으로 초기화
            nn.init.uniform_(param, -0.1, 0.1)

    #LSTM의 순전파
    def forward(self, x, hidden):
        """
        x: 현재 입력 벡터
        hidden: 이전 타임스텝의 은닉 상태(h_prev)와 셀 상태(c_prev)
        h_prev: 직전의 은닉 상태
        c_prev: 직전의 셀 상태
        """
        h_prev, c_prev = hidden

        #입력 게이트, 새로운 정보를 얼마나 받아들이지 결정, 시그모이드 함수로 0~1 사이의 값 출력, 1이면 새로운 정보 많이, 0이면 거의 받아들이지 않음음
        i_t = torch.sigmoid(x @ self.W_ii.T + h_prev @ self.W_hi.T + self.b_i)
        #망각 게이트, 이전 셀 상태를 얼마나 유지할지 결정, 시그모이드 함수로 0~1 사이의 값 출력, 1이면 기존 정보 유지, 0이면 삭제제
        f_t = torch.sigmoid(x @ self.W_if.T + h_prev @ self.W_hf.T + self.b_f)
        #셀 상태 후보값, 새로운 셀 상태 후보값을 계산, tanh 함수로 -1~1 사이의 값 출력
        g_t = torch.tanh(x @ self.W_ig.T + h_prev @ self.W_hg.T + self.b_g)
        #출력 게이트, 최종 은닉 상태에 반영할 정보를 결정, 시그모이드 함수로 0~1 사이의 값 출력
        o_t = torch.sigmoid(x @ self.W_io.T + h_prev @ self.W_ho.T + self.b_o)
        
        #새로운 셀 상태 c_t, 이전 셀 상태(c_prev)를 f_t(망각 게이트)로 조절, 새로운 정보 g_t를 i_t(입력 게이트)로 조절하여 반영영
        c_t = f_t * c_prev + i_t * g_t
        #c_t에 출력 게이트(o_t)를 곱하여 최종 은닉 상태 결정정
        h_t = o_t * torch.tanh(c_t)
        

        #새로운 은닉 상태 h_t와 셀 상태(c_t) 반환
        #h_t는 다음 레이어의 입력으로 사용
        #(h_t, c_t)는 다음 타임스텝에 입력될 은닉 상태로 사용
        return h_t, (h_t, c_t)

In [3]:
class LSTM(nn.Module):
    def __init__(self, input_window_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cells = nn.ModuleList([LSTMCell(input_window_size, hidden_size) if i == 0 
                                    else LSTMCell(hidden_size, hidden_size) 
                                    for i in range(num_layers)])
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        h = [torch.zeros(batch_size, self.hidden_size).to(x.device) for _ in range(self.num_layers)]
        c = [torch.zeros(batch_size, self.hidden_size).to(x.device) for _ in range(self.num_layers)]
        
        for t in range(seq_len):
            x_t = x[:, t, :]
            for i, cell in enumerate(self.cells):
                h[i], (h[i], c[i]) = cell(x_t, (h[i], c[i]))
                x_t = h[i]
        
        out = self.fc(h[-1])
        return out


In [4]:
#Example Usage
input_window_size=10
hidden_size=50
num_layers=2

model=LSTM(input_window_size, hidden_size, num_layers)
print(model)
sample_input=torch.randn(1, 5, input_window_size)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sample_input=sample_input.to(device)
output=model(sample_input) # shape=(1, )

LSTM(
  (cells): ModuleList(
    (0-1): 2 x LSTMCell()
  )
  (fc): Linear(in_features=50, out_features=1, bias=True)
)
